### RDKit Conformer Browser
Derived from Greg Landrum's code by Malitha Humayun Kabir<br>
As a part of GSoC 2017 <br>
Project : RDKit - 3Dmol.js integration <br>
Mentors: Paul Czodrowski and Greg Landrum <br>
Date: 13th August 2017 <br>
Acknowledgement: Peter Gedeck reviewed the codes, wrote initial class definition for molecule handling, and provided advice to restructure codes found in IPythonConsoleIntegration.py

Unofficial links (might help in code review processs): <br>
IPythonConsoleIntegration.py: 
https://github.com/malithakabir/notebook/blob/master/IPythonConsoleIntegration.py <br>
ChemUtils.py: https://github.com/malithakabir/notebook/blob/master/ChemUtils.py

In [1]:
%%html
<script href="https://raw.githubusercontent.com/3dmol/3Dmol.js/master/release/3Dmol.js"></script>

In [2]:
import IPythonConsoleIntegration as ipy
import ChemUtils
from rdkit import Chem
from rdkit.Chem import AllChem
from six.moves import urllib

### Example 1 : Extract ligand and show protein and ligand together

In [3]:
url = 'https://raw.githubusercontent.com/malithakabir/notebook/master/4i22.pdb'
response = urllib.request.urlopen(url)
data = response.read()
Mol = Chem.MolFromPDBBlock(data)
Mol is None

False

In [4]:
nm = ChemUtils.ExtractMolFragment(Mol,ResName = 'IRE')
nm

ExtractResult(match=<rdkit.Chem.rdchem.Mol object at 0x7f743c087910>, rest=<rdkit.Chem.rdchem.Mol object at 0x7f743c0877c0>)

#### nm contains two elements. nm.match is the ligand and nm.rest is the protein

In [5]:
params = AllChem.ETKDG()
params.numThreads=3
AllChem.EmbedMultipleConfs(nm.match,numConfs=10,params=params)

In [6]:
ipy.ShowConformers3D(uid = 'a', moldict = {'m0':nm.match}, protein = nm.rest,
                     useDrawAs=True, 
                     drawAs = 'stick', pStyle = 'cartoon',
                     propPanel = False, colorPanel = True, labelPanel = False)

In [7]:
ipy.ChangeActiveLigand(uid='a', molId='m0', confId=5, removeAllModels = True)

### Example 2 : Manually generated small molecules (without protein)

In [8]:
# Codes from Greg Landrum
# obj ms will have no pre calculated propery
data = [('m1','COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m2','COc1ccc2[nH]c([S@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m3','COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m4','CCOc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OCCC)c3C)nc2c1')]
ms = [(x,Chem.AddHs(Chem.MolFromSmiles(y))) for x,y in data]
params = AllChem.ETKDG()
params.numThreads=3
for nm,m in ms:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)
    # align to one of the ring systems:
    AllChem.AlignMolConformers(m,m.GetSubstructMatch(Chem.MolFromSmarts('c1[nH]c2ccccc2n1')))

#### Descriptive uid is possible but it must not contain space and/or special characters 

In [9]:
ipy.ShowConformers3D(uid='smallMolecules', moldict=dict(ms), useDrawAs=False, drawAs = 'line', 
                     propPanel = True, calculateProperties = True, 
                     colorPanel = False, labelPanel = False)

In [10]:
ipy.ChangeActiveLigand(uid='smallMolecules', molId='m3', confId=7, removeAllModels = True)

### Example 3 : Show small molecules with descriptors

In [11]:
# Codes from Paolo Tosco (Slightly modified version)
# obj ms2 will have some pre calculated properties
url = 'https://github.com/rdkit/rdkit/raw/master/Docs/Book/data/cdk2.sdf'
response = urllib.request.urlopen(url)
data = response.read()
suppl = Chem.SDMolSupplier()
suppl.SetData(data)
ms2=[mol for mol in suppl]
params = AllChem.ETKDG()
params.numThreads=3
for m in ms2:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)


#### dict keys must be string type. 

In [12]:
mols=[(str(ids),mol) for ids,mol in enumerate(ms2)]
mols_2 = dict(mols)
print(type(list(mols_2.keys())[0]))

<type 'str'>


#### user may not supply uid. In this case uid will be generated automatically

In [13]:
ipy.ShowConformers3D(moldict=mols_2, useDrawAs=True, 
                     propPanel = True, calculateProperties = False,
                     colorPanel = True, labelPanel = True)

#### Please see the top right corner of the above viewer and replace the uid from the following line

In [14]:
ipy.ChangeActiveLigand(uid='150257394167', molId='3', confId=9, removeAllModels = True)

### Thank you you very much for your time. Have a great day! 